**Water Polygon Data Cleaning**

Let's clean and merge polygon-related data from the National Hydrography Dataset.

In [1]:
%run ../bootstrap.py
setup_project_path()

from scripts.io_helpers import export_interim, read_interim_layer
from scripts.geometry_helpers import strip_z_polygon, drop_missing_geometry, validate_geometry
from scripts import data_config as dc
import geopandas as gpd
import fiona

Let's take a look at the layers.

In [2]:
gdb_path = dc.RAW_DATA_PATH / "NHDPlus_H_National_Release_2_GDB" / "NHDPlus_H_National_Release_2.gdb"

layers = fiona.listlayers(gdb_path)
for layer in layers:
    print(layer)

NHDPlusGageSmooth
NHDPlusFlow
NHDArea
NHDLine
NHDPlusBoundaryUnit
NHDPlusCatchment
NHDPlusGage
NHDPlusSink
NHDPlusWall
NHDPoint
NHDWaterbody
NonNetworkNHDFlowline
WBDHU12
NHDPlusConnect
NetworkNHDFlowline


For polygon data, we want to merge NetworkNHDFlowline and NonNetworkNHDFlowline. Since these are massive datasets, let's load our buffered CO state boundary and use it to mask the data loading. This will save on time and memory, and the extra step of intersecting later on.

First, let's check what CRS the NHD data is in by reading a few rows, and project the state buffer to that CRS:

In [3]:
crs_check_area = gpd.read_file(dc.RAW_FILES["nhd"], layer="NHDArea", rows=10)
crs_check_non_body = gpd.read_file(dc.RAW_FILES["nhd"], layer="NHDWaterbody", rows=10)
print("Network CRS: ", crs_check_area.crs)
print("Non network CRS: ", crs_check_non_body.crs)

Network CRS:  COMPD_CS["NAD83 + NAVD88 height",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4269"]],VERT_CS["NAVD88 height",VERT_DATUM["North American Vertical Datum 1988",2005,AUTHORITY["EPSG","5103"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Gravity-related height",UP],AUTHORITY["EPSG","5703"]]]
Non network CRS:  COMPD_CS["NAD83 + NAVD88 height",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4269"]],VERT_CS["NAVD88 height",VERT_DATUM["North American Vert

The NHD data uses EPSG:4269 for horizontal coordinates, so let's load project the buffered CO border to this CRS.

In [4]:
# Load buffered CO boundary
co_mask = read_interim_layer("state_boundary_buffered")
# Project to NHD horizontal CRS
co_mask.to_crs("EPSG:4269")
co_mask.head()


,NAME,geometry
0,Colorado,"POLYGON ((143525.677 4102169.628, 143523.188 4..."


Now, let's load the NHD data with the CO mask.

In [5]:
# Load flowlines with CO mask
area = gpd.read_file(dc.RAW_FILES["nhd"], layer="NHDArea", mask=co_mask)
print("NHDArea Columns:", area.columns.tolist())
body = gpd.read_file(dc.RAW_FILES["nhd"], layer="NHDWaterbody", mask=co_mask)
print("NHDWaterbody Columns:", body.columns.tolist())

/Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/venv/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


NHDArea Columns: ['permanent_identifier', 'fdate', 'resolution', 'gnis_id', 'gnis_name', 'areasqkm', 'elevation', 'ftype', 'fcode', 'visibilityfilter', 'nhdplusid', 'vpuid', 'onoffnet', 'purpcode', 'burn', 'Shape_Length', 'Shape_Area', 'geometry']
NHDWaterbody Columns: ['permanent_identifier', 'fdate', 'resolution', 'gnis_id', 'gnis_name', 'areasqkm', 'elevation', 'reachcode', 'ftype', 'fcode', 'visibilityfilter', 'nhdplusid', 'vpuid', 'onoffnet', 'purpcode', 'burn', 'Shape_Length', 'Shape_Area', 'geometry']


Let's see what columns are shared by both, and trim down the columns for merging.

In [6]:
shared_cols = [col for col in area.columns if col in body.columns]
print(shared_cols)

['permanent_identifier', 'fdate', 'resolution', 'gnis_id', 'gnis_name', 'areasqkm', 'elevation', 'ftype', 'fcode', 'visibilityfilter', 'nhdplusid', 'vpuid', 'onoffnet', 'purpcode', 'burn', 'Shape_Length', 'Shape_Area', 'geometry']


In [7]:
trimmed_cols = [
    'permanent_identifier', 'gnis_name', 'fcode', 
    'nhdplusid', 'Shape_Length', 'Shape_Area', 'geometry'
    ]

area_trimmed = area.copy()[trimmed_cols]
body_trimmed = body.copy()[trimmed_cols]

Before merging, let's add a column to signify the original type - NHDArea or NHDWaterbody.

In [8]:
area_trimmed['NHDType'] = 'NHDArea'
body_trimmed['NHDType'] = 'NHDWaterbody'
print(body_trimmed.loc[0])

permanent_identifier                                            132228513
gnis_name                                                            None
fcode                                                               39004
nhdplusid                                                23001800227068.0
Shape_Length                                                     0.001198
Shape_Area                                                            0.0
geometry                MULTIPOLYGON Z (((-106.17111227897482 40.99343...
NHDType                                                      NHDWaterbody
Name: 0, dtype: object


**Merging**

Ready to merge! Now that columns are shared, let's combine NHDArea and NHDWaterbody.

In [9]:
import pandas as pd

polygons_merged = gpd.GeoDataFrame(
    pd.concat([area_trimmed, body_trimmed], ignore_index=True),
    crs=area_trimmed.crs
)

print("NHDTypes: ", polygons_merged['NHDType'].unique())
polygons_merged.head()

NHDTypes:  ['NHDArea' 'NHDWaterbody']


,permanent_identifier,gnis_name,fcode,nhdplusid,Shape_Length,Shape_Area,geometry,NHDType
0,128861871,None,34306,2.300180e+13,0.001405,5.508030e-08,"MULTIPOLYGON Z (((-106.31636 40.50731 0, -106....",NHDArea
1,128867328,None,46006,2.300180e+13,3.109968,3.198720e-04,"MULTIPOLYGON Z (((-106.40743 40.72295 0, -106....",NHDArea
2,128859139,None,34306,2.300180e+13,0.002341,1.658604e-07,"MULTIPOLYGON Z (((-106.08472 40.61818 0, -106....",NHDArea
3,132228795,None,46006,2.300180e+13,5.435655,6.241810e-04,"MULTIPOLYGON Z (((-105.82683 41.15492 0, -105....",NHDArea
4,128861873,None,34306,2.300180e+13,0.002711,3.738544e-07,"MULTIPOLYGON Z (((-106.29214 40.55282 0, -106....",NHDArea


**Geometry**

Almost there. Now that we've merged the two water polygon datasets, let's check the geometry types:

In [10]:
print("Geometry types: ", polygons_merged.geometry.type.unique())
print("Has Z axis: ", polygons_merged.geometry.apply(lambda z: z.has_z).value_counts())

Geometry types:  ['MultiPolygon']
Has Z axis:  geometry
True    118258
Name: count, dtype: int64


All geometries are of type MultiLineString, and contain a Z axis. We don't really need the Z axis for our purposes, so let's strip it for efficiency.

In [11]:
polygons_merged['geometry'] = polygons_merged['geometry'].apply(strip_z_polygon)
print("Has Z axis: ", polygons_merged.geometry.apply(lambda z: z.has_z).value_counts())
polygons_merged.head()

Has Z axis:  geometry
False    118258
Name: count, dtype: int64


,permanent_identifier,gnis_name,fcode,nhdplusid,Shape_Length,Shape_Area,geometry,NHDType
0,128861871,None,34306,2.300180e+13,0.001405,5.508030e-08,"MULTIPOLYGON (((-106.31636 40.50731, -106.3162...",NHDArea
1,128867328,None,46006,2.300180e+13,3.109968,3.198720e-04,"MULTIPOLYGON (((-106.40743 40.72295, -106.4078...",NHDArea
2,128859139,None,34306,2.300180e+13,0.002341,1.658604e-07,"MULTIPOLYGON (((-106.08472 40.61818, -106.0858...",NHDArea
3,132228795,None,46006,2.300180e+13,5.435655,6.241810e-04,"MULTIPOLYGON (((-105.82683 41.15492, -105.8271...",NHDArea
4,128861873,None,34306,2.300180e+13,0.002711,3.738544e-07,"MULTIPOLYGON (((-106.29214 40.55282, -106.2918...",NHDArea


Finally, let's drop missing geometries and validate.

In [12]:
print("Rows before validation: ", polygons_merged.shape[0])
polygons_validated = drop_missing_geometry(polygons_merged)
polygons_validated = validate_geometry(polygons_merged)
print("Rows after validation: ", polygons_validated.shape[0])

Rows before validation:  118258
Rows after validation:  118233


**Exporting Data**

Our flowline data is fully cleaned and merged! Let's export to data/interim for further use. This file (water_polygon_clean) is marked for display, so will also be exported to data/processed for display in visualization.

In [13]:
export_interim(polygons_validated, "water_polygon_clean", driver="GPKG", verbose=True)

Saved to interim: /Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/data/interim/water_polygon_clean.gpkg
Also saved to processed: /Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/data/processed/water_polygon_clean.gpkg
